## Data organization

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [17]:
data = pd.read_csv('data.csv')
df = data.copy()
df = df.sample(frac=1).reset_index(drop=True)
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Guilty,0.523684,0.450804,-0.995983,0.999972,0.552573,0.373893,-0.930482,0.999962,0.569156,...,-0.011148,0.0,0.582584,0.369692,0.009396,0.0,0.586948,0.365601,0.009531,0.0
1,Armenia,0.486007,0.431773,-1.232248,0.999901,0.522058,0.360225,-1.145637,0.999764,0.546086,...,-0.002062,0.0,0.566201,0.356816,0.018572,0.0,0.571520,0.351379,0.018888,0.0
2,Star,0.425853,0.537240,-0.504568,0.999924,0.440406,0.489240,-0.413601,0.999838,0.452841,...,-0.004900,0.0,0.486329,0.501907,0.015526,0.0,0.488939,0.497582,0.016310,0.0
3,Prayer,0.236152,0.510734,-0.409190,0.999872,0.262398,0.457627,-0.483586,0.999909,0.272496,...,-0.012548,0.0,0.282198,0.468556,-0.040394,0.0,0.287529,0.461740,-0.043540,0.0
4,Prayer,0.556235,0.456706,-0.431842,0.999108,0.549914,0.395114,-0.373876,0.998768,0.557004,...,0.000615,0.0,0.577383,0.436700,0.041680,0.0,0.579086,0.431563,0.044625,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,Prayer,0.242467,0.513444,-0.407843,0.999875,0.268420,0.459530,-0.476899,0.999910,0.278648,...,-0.013224,0.0,0.287756,0.468318,-0.040715,0.0,0.293186,0.460747,-0.043772,0.0
1601,Prayer,0.325390,0.512819,-0.596722,0.999663,0.357311,0.454009,-0.564050,0.999372,0.376336,...,-0.009591,0.0,0.392010,0.463658,-0.019936,0.0,0.397553,0.457777,-0.021900,0.0
1602,Muscles,0.451752,0.568155,-0.551326,0.999811,0.474012,0.531654,-0.492730,0.999550,0.486103,...,-0.000960,0.0,0.500277,0.524583,0.013379,0.0,0.502905,0.521358,0.013907,0.0
1603,Sheytan,0.454950,0.429795,-0.852592,0.999923,0.484224,0.378463,-0.792749,0.999818,0.501182,...,0.007828,0.0,0.517085,0.372704,0.030966,0.0,0.521985,0.367826,0.031963,0.0


In [18]:
X = df.drop('class', axis=1)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
y_train

1397    Sheytan
19      Muscles
1324     Guilty
784        Star
957      Guilty
         ...   
74       Prayer
58      Albania
195     Armenia
448      Prayer
500     Albania
Name: class, Length: 1284, dtype: object

## Scikit learn models

In [30]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

model = make_pipeline(StandardScaler(), GradientBoostingClassifier())

In [31]:
trained_model = model.fit(X_train, y_train)

In [35]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

with open('best_model.pkl', 'wb') as f:
    pickle.dump(model, f)

## Tensorflow model

## Evaluation

In [36]:
from sklearn.metrics import accuracy_score
import mediapipe as mp
import cv2

y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)
mp_drawing = mp.solutions.drawing_utils
mp_hol = mp.solutions.holistic

In [39]:
with open('best_model.pkl', 'wb') as f: #'wb' means it will be written binary
    pickle.dump(model, f)
import warnings
warnings.filterwarnings("ignore")

In [40]:
cap = cv2.VideoCapture(0)

with mp_hol.Holistic(min_detection_confidence=0.5,
                     min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #Recolor feed
        image.flags.writeable = False #prevent copying image data + able to use the same image for rendering
        
        results = holistic.process(image)  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_hol.FACEMESH_TESSELATION,
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 ) 
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hol.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        #left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hol.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        #pose 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_hol.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        #export coord
        try:
            #for pose
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[coords.x, coords.y, coords.z, coords.visibility]for coords in pose]).flatten())
            
            #for face
            face = results.face_landmarks.landmark
            face_row = list(np.array([[coords.x, coords.y, coords.z, coords.visibility]for coords in face]).flatten()) 
            
            #adding a row in csv file + giving a label to
            ligne = pose_row + face_row
            X = pd.DataFrame([ligne])
            body_lang_class = model.predict(X)[0]
            body_lang_prob = model.predict_proba(X)[0]
            print(body_lang_class, body_lang_prob)
            
        except:
            pass
        
        
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Muscles [9.84394036e-08 2.38719658e-07 1.52135951e-05 9.99937822e-01
 8.16318432e-07 3.70173361e-05 1.30177784e-07 8.66305904e-06]
Muscles [1.39880327e-07 3.39209598e-07 3.65726812e-05 9.99902964e-01
 3.35762779e-07 4.47809743e-05 1.84928300e-07 1.46829632e-05]
Muscles [1.36288763e-07 3.93699575e-07 4.24484510e-05 9.99932703e-01
 3.27159660e-07 9.50521983e-06 1.80180080e-07 1.43059638e-05]
Muscles [1.36295046e-07 3.93716172e-07 5.06984883e-05 9.99924702e-01
 3.27174743e-07 9.25535608e-06 1.80187799e-07 1.43066233e-05]
Muscles [1.36279231e-07 3.93670487e-07 1.76195879e-04 9.99808675e-01
 3.27136778e-07 9.25428213e-06 1.80166891e-07 4.83764474e-06]
Muscles [2.51285243e-07 6.09363958e-07 5.68062305e-04 9.99393530e-01
 6.03207432e-07 1.02053723e-05 3.32210732e-07 2.64065719e-05]
Muscles [3.55112282e-07 8.61124863e-07 9.56435588e-04 9.98989287e-01
 8.52443085e-07 1.44220687e-05 4.69475645e-07 3.73173447e-05]
Muscles [3.55112282e-07 8.61124863e-07 9.56435588e-04 9.98989287e-01
 8.52443085e-0

QLF [2.74478059e-05 2.80077532e-05 4.01104761e-05 4.24510439e-05
 4.32746355e-05 9.99724822e-01 3.61442361e-05 5.77417942e-05]
QLF [2.74478055e-05 2.80077528e-05 4.01104756e-05 4.24510433e-05
 4.32746349e-05 9.99724808e-01 3.61584427e-05 5.77417934e-05]
QLF [2.74477586e-05 2.80077050e-05 4.01104071e-05 4.24509708e-05
 4.32745610e-05 9.99723101e-01 3.61583810e-05 5.94489943e-05]
QLF [2.74478055e-05 2.80077528e-05 4.01104756e-05 4.24510433e-05
 4.32746349e-05 9.99724808e-01 3.61584427e-05 5.77417934e-05]
QLF [2.74478055e-05 2.80077528e-05 4.01104756e-05 4.24510433e-05
 4.32746349e-05 9.99724808e-01 3.61584427e-05 5.77417934e-05]
QLF [2.74477990e-05 2.80077462e-05 4.01104661e-05 4.24510333e-05
 4.32746246e-05 9.99724572e-01 3.61584342e-05 5.79783660e-05]
QLF [4.25623281e-05 4.34306184e-05 6.21978768e-05 6.58273113e-05
 6.71044252e-05 9.99573270e-01 5.60696011e-05 8.95381294e-05]
QLF [4.25622155e-05 4.34305034e-05 6.21977121e-05 6.58271370e-05
 6.71042475e-05 9.99570623e-01 5.60694526e-05 

QLF [3.45658354e-07 3.52640982e-07 5.05024363e-07 5.34494064e-07
 9.33839133e-07 9.99996143e-01 4.55264971e-07 7.29996188e-07]
QLF [3.98804298e-07 4.01451619e-07 5.74927074e-07 6.08475810e-07
 1.06309604e-06 9.99995604e-01 5.18280259e-07 8.31038270e-07]
QLF [9.76835753e-07 9.96677573e-07 1.42736234e-06 1.51065325e-06
 2.63933169e-06 9.99989099e-01 1.28672619e-06 2.06320554e-06]
QLF [9.89977516e-07 9.96657674e-07 1.42733384e-06 1.51062309e-06
 2.63927900e-06 9.99989086e-01 1.28670050e-06 2.06316435e-06]
QLF [6.18850489e-07 6.23026421e-07 8.92248880e-07 9.44314304e-07
 1.64985490e-06 9.99993178e-01 8.04336761e-07 1.28971655e-06]
QLF [3.98812255e-07 4.01459629e-07 5.74938545e-07 6.08487950e-07
 1.06311725e-06 9.99995604e-01 5.18290600e-07 8.31054851e-07]
QLF [3.93510234e-07 4.01459568e-07 5.74938458e-07 6.08487858e-07
 1.21962475e-06 9.99995453e-01 5.18290521e-07 8.31054725e-07]
QLF [3.93510234e-07 4.01459568e-07 5.74938458e-07 6.08487858e-07
 1.21962475e-06 9.99995453e-01 5.18290521e-07 

QLF [5.47381200e-05 5.63164229e-05 8.06519010e-05 8.53581837e-05
 2.46196865e-04 9.99181808e-01 7.27053744e-05 2.22225101e-04]
QLF [5.47437339e-05 5.63221987e-05 8.06601727e-05 8.53669380e-05
 1.43662334e-04 9.99284284e-01 7.27128310e-05 2.22247893e-04]
QLF [4.47353743e-05 4.60252609e-05 6.59137175e-05 6.97599825e-05
 1.17397697e-04 9.99415133e-01 5.94193248e-05 1.81616085e-04]
QLF [4.47353743e-05 4.60252609e-05 6.59137175e-05 6.97599825e-05
 1.17397697e-04 9.99415133e-01 5.94193248e-05 1.81616085e-04]
QLF [5.47437339e-05 5.63221987e-05 8.06601727e-05 8.53669380e-05
 1.43662334e-04 9.99284284e-01 7.27128310e-05 2.22247893e-04]
QLF [5.47381200e-05 5.63164229e-05 8.06519010e-05 8.53581837e-05
 2.46196865e-04 9.99181808e-01 7.27053744e-05 2.22225101e-04]
QLF [5.47381200e-05 5.63164229e-05 8.06519010e-05 8.53581837e-05
 2.46196865e-04 9.99181808e-01 7.27053744e-05 2.22225101e-04]
QLF [5.47381200e-05 5.63164229e-05 8.06519010e-05 8.53581837e-05
 2.46196865e-04 9.99181808e-01 7.27053744e-05 

Star [3.73161708e-07 3.80774386e-07 5.45314786e-07 5.77135556e-07
 5.88332547e-07 1.21735959e-05 4.91585631e-07 9.99984870e-01]
Star [0.00674957 0.00688726 0.00986339 0.01043894 0.01064147 0.22019001
 0.00889156 0.72633781]
QLF [0.00542315 0.00553378 0.00792504 0.00838749 0.00855022 0.89519921
 0.0071442  0.0618369 ]
QLF [0.02031038 0.02072472 0.0296803  0.03141223 0.03202166 0.67223882
 0.02675594 0.16685595]
QLF [0.00439533 0.004485   0.00642306 0.00679786 0.00692975 0.71511542
 0.0057902  0.25006337]
QLF [0.00498249 0.00508413 0.00728109 0.00770596 0.00785547 0.81064512
 0.00656369 0.14988204]
QLF [0.00532653 0.0054352  0.00778386 0.00823807 0.0083979  0.86662108
 0.00701693 0.09118044]
Star [1.94987863e-07 1.98965735e-07 2.84942898e-07 3.01570180e-07
 3.07420935e-07 1.84004267e-05 2.56867846e-07 9.99980055e-01]
Star [6.58396649e-05 6.78881880e-05 9.72240643e-05 2.05856248e-03
 1.04094879e-04 2.10720130e-03 8.76447040e-05 9.95411545e-01]
Star [0.01152346 0.01175613 0.01701631 0.0180

Muscles [5.45718635e-09 6.26928804e-09 2.42610940e-08 8.62278230e-01
 1.91826344e-08 9.12978361e-07 7.11307757e-09 1.37720795e-01]
Muscles [5.44839330e-09 5.26442460e-09 2.42640571e-08 8.62377502e-01
 1.91848428e-08 5.98931621e-08 7.11389648e-09 1.37622377e-01]
Star [2.78616766e-08 2.85310096e-08 4.72370281e-08 5.27893214e-06
 1.00860005e-07 8.75255656e-07 3.69225856e-08 9.99993604e-01]
Guilty [4.07474919e-06 4.09533731e-06 5.18825861e-01 8.05122850e-05
 1.28424815e-05 4.86519804e-05 5.34160442e-06 4.81018620e-01]
Muscles [8.32304744e-08 8.36509992e-08 1.24620918e-07 9.99997994e-01
 2.19769241e-07 4.80150948e-07 1.09107141e-07 9.05676303e-07]
Muscles [8.32345060e-08 8.45499638e-08 1.24626955e-07 9.99995814e-01
 1.93363538e-06 3.17247944e-07 1.09112426e-07 1.53378557e-06]
Muscles [3.43876437e-08 3.49314183e-08 5.14890370e-08 9.99998528e-01
 7.23748538e-07 1.75084368e-07 4.50792829e-08 4.06988851e-07]
Muscles [3.79734337e-06 3.82574026e-06 5.47892087e-06 9.95309113e-01
 6.78126300e-06 1.

Star [1.26152677e-08 1.27014933e-08 1.81900684e-08 1.18816655e-03
 2.25138564e-08 6.78553302e-08 1.63978247e-08 9.98811683e-01]
Star [1.52149155e-08 1.53189097e-08 2.19385239e-08 1.18395427e-03
 2.71533216e-08 2.79982608e-07 1.97769498e-08 9.98815666e-01]
Star [1.26155038e-08 1.27017310e-08 1.81904088e-08 1.16942310e-03
 2.25142777e-08 2.32148622e-07 1.63981315e-08 9.98830262e-01]
Star [5.56207306e-07 5.60008994e-07 8.02000333e-07 3.00073729e-01
 9.92636199e-07 1.02352440e-05 7.22980286e-07 6.99912402e-01]
Star [2.05284266e-08 2.06687388e-08 2.96001235e-08 2.27644377e-03
 3.66360872e-08 3.77761048e-07 2.66836620e-08 9.97723044e-01]
Star [2.05447597e-08 2.06851835e-08 2.96236744e-08 2.31104042e-03
 3.66652360e-08 3.78061607e-07 2.67048923e-08 9.97688447e-01]
Star [4.86785570e-08 4.90112759e-08 7.01900505e-08 2.62620716e-02
 8.68742594e-08 8.95775554e-07 6.32743164e-08 9.73736715e-01]
Star [2.03229515e-08 2.04618593e-08 2.93038472e-08 1.30824270e-02
 3.62693858e-08 3.73979926e-07 2.64165

Star [1.93701541e-08 8.95204844e-04 2.78737097e-08 2.95428261e-01
 4.76084177e-08 5.49980594e-05 2.46736224e-08 7.03621416e-01]
Star [2.34596519e-08 1.08420377e-03 3.37585093e-08 1.46676501e-01
 5.76596811e-08 6.66049049e-05 2.98828078e-08 8.52172545e-01]
Star [2.34596519e-08 1.08420377e-03 3.37585093e-08 1.46676501e-01
 5.76596811e-08 6.66049049e-05 2.98828078e-08 8.52172545e-01]
Star [2.48057873e-08 1.14641633e-03 3.56957391e-08 9.77338753e-02
 6.09682442e-08 4.86024938e-05 3.15975095e-08 9.01070953e-01]
Star [2.47622040e-08 1.15845443e-03 3.56330205e-08 9.69048440e-02
 6.98208527e-08 4.85170976e-05 3.15419917e-08 9.01888023e-01]
Star [2.28773390e-08 1.07027447e-03 3.29206839e-08 1.67864782e-01
 6.45061853e-08 4.48240428e-05 2.91410586e-08 8.31019970e-01]
Star [2.28773390e-08 1.07027447e-03 3.29206839e-08 1.67864782e-01
 6.45061853e-08 4.48240428e-05 2.93933515e-08 8.31019970e-01]
Star [2.28773390e-08 1.07027447e-03 3.29206839e-08 1.67864782e-01
 6.45061853e-08 4.48240428e-05 2.93933